# Generate word2vec model - include all new topic words

lemmatizing with UDPIPE is ~10x faster than with spaCy fi model

In [2]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [3]:
spacy_udpipe.download('fi')
nlp = spacy_udpipe.load('fi')

Already downloaded a model for the 'fi' language


In [4]:
# set variables 
LANG = 'fi-UPIPE'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

#temp change
LANG = 'fi'
model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'
LANG = 'fi-UPIPE'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [5]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [6]:
# process all words in fasttext model  takes ~3h for EL
all_words_lemmas = []

lemmas = {}
word_lemma = {}
words = model_big.index2word
for word in tqdm(words):
    # skip "words" which are not regular words
    if not re.fullmatch("([^\d\W])+", word):
        continue
    doc = nlp(word)
    lemma = doc[0].lemma_.lower()
    all_words_lemmas.append((word, lemma))
print("Number of words in big model: ", len(all_words_lemmas))

#save pickled dump 
with open(pickled_words_lemmas, 'wb') as f:
    pickle.dump(all_words_lemmas, f)

100%|██████████| 2000000/2000000 [08:17<00:00, 4018.76it/s]


Number of words in big model:  1746550


In [7]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('ja', 'ja'),
 ('on', 'olla'),
 ('oli', 'olla'),
 ('ei', 'ei'),
 ('että', 'että'),
 ('myös', 'myös'),
 ('tai', 'tai'),
 ('Lisätiedot', 'lisä#tieto'),
 ('ovat', 'olla'),
 ('mutta', 'mutta'),
 ('joka', 'joka'),
 ('se', 'se'),
 ('sekä', 'sekä'),
 ('n', 'n'),
 ('vuonna', 'vuosi'),
 ('kun', 'kun'),
 ('hän', 'hän'),
 ('sen', 'se'),
 ('ole', 'olla'),
 ('kuin', 'kuin'),
 ('niin', 'niin'),
 ('kanssa', 'kanssa'),
 ('voi', 'voida'),
 ('kello', 'kello'),
 ('Hän', 'hän'),
 ('mukaan', 'mukaan'),
 ('jälkeen', 'jälkeen'),
 ('klo', 'kello'),
 ('jo', 'jo'),
 ('Suomen', 'suomi'),
 ('vain', 'vain'),
 ('s', 's'),
 ('ollut', 'olla'),
 ('olla', 'olla'),
 ('p', 'p'),
 ('vuoden', 'vuosi'),
 ('jossa', 'joka'),
 ('sitä', 'se'),
 ('jonka', 'joka'),
 ('_', '_'),
 ('jotka', 'joka'),
 ('kuitenkin', 'kuitenkin'),
 ('vielä', 'vielä'),
 ('jos', 'jos'),
 ('nyt', 'nyt'),
 ('eli', 'eli'),
 ('of', 'of'),
 ('sitten', 'sitten'),
 ('noin', 'noin'),
 ('Se', 'se')]


In [8]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

899403
1746550


/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [9]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('olla', 322),
 ('tehdä', 191),
 ('käyttää', 176),
 ('antaa', 175),
 ('saada', 174),
 ('ottaa', 162),
 ('se', 159),
 ('tulla', 159),
 ('hyvä', 159),
 ('pitää', 153),
 ('minä', 151),
 ('suuri', 137),
 ('sanoa', 133),
 ('haluta', 132),
 ('katsoa', 131),
 ('ei', 130),
 ('odottaa', 128),
 ('ostaa', 123),
 ('puhua', 123),
 ('kirjoittaa', 119)]

In [10]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


154749 154749


In [11]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [12]:
# test the model
pprint(new_vector_space.most_similar('isä'))
pprint(new_vector_space.most_similar_cosmul(positive=['nainen', 'kuningas'], negative=['mies']))

[('äiti', 0.8445054888725281),
 ('poika', 0.8265360593795776),
 ('äidi', 0.7837938666343689),
 ('poja', 0.7776297330856323),
 ('isi', 0.7486921548843384),
 ('velji', 0.7473832368850708),
 ('isoisä', 0.7441078424453735),
 ('tytär', 0.7296373844146729),
 ('iskä', 0.7261735796928406),
 ('sisko', 0.7250024080276489)]
[('kuninkas', 0.904109001159668),
 ('kuninkai', 0.8895664215087891),
 ('kuninkaa', 0.8871010541915894),
 ('kuningatar', 0.8829861283302307),
 ('kuningattari', 0.8534073829650879),
 ('hallitsija', 0.8524658679962158),
 ('kuninkaallinen', 0.8499844670295715),
 ('kuninka', 0.8450592160224915),
 ('kuninkuus', 0.8370404243469238),
 ('kuningattar', 0.8357623815536499)]


In [13]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [14]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topics_nolemma)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  1%|          | 60/5305 [00:00<00:08, 591.30it/s]

5305


 84%|████████▎ | 4437/5305 [00:02<00:00, 1818.87it/s]

lemmas = 0  ['paikallinen kehitys\n', 'maaseuturakentaminen\n', 'vesitalous\n', 'koulutustaso\n', 'yleissivistävä koulutus\n', 'rantapenger\n', 'yhteisöoikeus\n', 'kulttuuriperintö\n', 'viljelysmaa, maatalousmaa\n', 'maatalousmaisema\n', 'maataloustuotanto\n', 'taloudellinen kehitys\n', 'ravintoketju\n', 'elintarviketeollisuus\n', 'ruuan laatu\n', 'globaali lämpeneminen\n', 'kasvihuoneilmiö\n', 'elinkeinoelämän kehitys\n', 'työmarkkinat\n', 'pääsy maa-alueelle\n', 'uusjako, tilusten sijoitus\n', 'maankäytön suunnittelu\n', 'maisema-arkkitehtuuri\n', 'maisemansuunnittelu\n', 'maankäyttö\n', 'kaavoitus, maankäytön suunnittelu\n', 'meriympäristö\n', 'luonnonsuojelu\n', 'luonnonsuojelu\n', 'biodynaaminen viljely\n', 'fyysinen suunnittelu\n', 'alkeisopetus, perusasteen koulutus\n', 'autoteollisuus\n', 'elämänlaatu\n', 'aluekehitys\n', 'resurssijohtaminen\n', 'maaseutuväestö\n', 'keskiasteen koulutus\n', 'maan tiivistys\n', 'maan kasvukunnon heikkeneminen\n', 'maaeroosio\n', 'kestävä kehitys

 94%|█████████▍| 4992/5305 [00:02<00:00, 1817.50it/s]

lemmas = 0  ['paikallinen kehitys\n', 'maaseuturakentaminen\n', 'vesitalous\n', 'koulutustaso\n', 'yleissivistävä koulutus\n', 'rantapenger\n', 'yhteisöoikeus\n', 'kulttuuriperintö\n', 'viljelysmaa, maatalousmaa\n', 'maatalousmaisema\n', 'maataloustuotanto\n', 'taloudellinen kehitys\n', 'ravintoketju\n', 'elintarviketeollisuus\n', 'ruuan laatu\n', 'globaali lämpeneminen\n', 'kasvihuoneilmiö\n', 'elinkeinoelämän kehitys\n', 'työmarkkinat\n', 'pääsy maa-alueelle\n', 'uusjako, tilusten sijoitus\n', 'maankäytön suunnittelu\n', 'maisema-arkkitehtuuri\n', 'maisemansuunnittelu\n', 'maankäyttö\n', 'kaavoitus, maankäytön suunnittelu\n', 'meriympäristö\n', 'luonnonsuojelu\n', 'luonnonsuojelu\n', 'biodynaaminen viljely\n', 'fyysinen suunnittelu\n', 'alkeisopetus, perusasteen koulutus\n', 'autoteollisuus\n', 'elämänlaatu\n', 'aluekehitys\n', 'resurssijohtaminen\n', 'maaseutuväestö\n', 'keskiasteen koulutus\n', 'maan tiivistys\n', 'maan kasvukunnon heikkeneminen\n', 'maaeroosio\n', 'kestävä kehitys

100%|██████████| 5305/5305 [00:03<00:00, 1691.59it/s]

5305


In [17]:
print(len(topics),len(topics_nolemma))  

5305 5305


In [18]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 rantapenger
l2 yhteisöoikeus
l2 maatalousmaisema
w1 uusja
23 needs to be added manually:  maisemansuunnittelu  -  maisemansuunnittelu  -  maisemansuunnittelu
35 needs to be added manually:  resurssijohtaminen  -  resurssijohtaminen  -  resurssijohtaminen
40 needs to be added manually:  maaeroosio  -  maaeroosio  -  maaeroosio
l2 vesieroosio
l2 tuulieroosio
l2 sisävesiliikenne
w2 ympäristö#muutos
63 needs to be added manually:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus
64 needs to be added manually:  maaseutusuunnittelu  -  maaseutusuunnittelu  -  maaseutusuunnittelu
w3 luonnon#varainen
l2 luonnonsuojelujärjestö
76 needs to be added manually:  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus  -  ruuanjalostusteollisuus
l2 asutus#alue
83 needs to be added manually:  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka  -  luonnonsuojelupolitiikka
l3 eurooppaa
l2 luonnonsuojeluohjelma
l2 luonnonsuojelulainsäädäntö
96 needs to be added 

585 needs to be added manually:  maannos#laji  -  maannoslaji  -  maalaji maannoslaji
589 needs to be added manually:  jäähdytysteollisuus  -  jäähdytysteollisuus  -  jäähdytysteollisuus
l2 sokeriteollisuus
599 needs to be added manually:  kehityskata  -  kehityskaava  -  kehityskaava
600 needs to be added manually:  maaperäbiologia  -  maaperäbiologia  -  maaperäbiologia
607 needs to be added manually:  keinokasteluviljely  -  keinokasteluviljely  -  keinokasteluviljely
l2 merioikeus
l2 murtovesi
618 needs to be added manually:  puolisuolainen  -  puolisuolainen  -  murtovesi puolisuolainen vesi
619 needs to be added manually:  kasviperintö  -  kasviperintö  -  kasviperintö
l2 säteilyvaikutus
627 needs to be added manually:  vesiorganismi  -  vesiorganismi  -  vesiorganismi
630 needs to be added manually:  ympäristönseuranta  -  ympäristönseuranta  -  ympäristönseuranta
635 needs to be added manually:  johtamissuunnitelma  -  johtamissuunnitelma  -  hallintasuunnitelma johtamissuunnit

957 needs to be added manually:  poistolämpö  -  poistolämpö  -  hukkalämpö poistolämpö jätelämpö
l2 jätelämpö
l2 ympäristökustannus
969 needs to be added manually:  tehokkuuskriteeri  -  tehokkuuskriteeri  -  tehokkuuskriteeri
970 needs to be added manually:  laatutodistus  -  laatutodistus  -  laatutodistus
l2 alue#vesi
l2 vesikaivo
976 needs to be added manually:  ympäristökertomus  -  ympäristökertomus  -  ympäristökertomus
l2 pakkausteollisuus
l2 alivirtaama
984 needs to be added manually:  jalostustekniikka  -  jalostustekniikka  -  jalostustekniikka
l2 ympäristö#talous
l2 laajentumispolitiikka
l2 asiantuntijajärjestelmä
990 needs to be added manually:  alkali#maa  -  alkalimaa  -  emäksinen maa alkalimaa
991 needs to be added manually:  ravinto#taso  -  ravintotaso  -  ravintotaso
993 needs to be added manually:  geomorfinen  -  geomorfinen  -  geomorfinen prosessi
l2 jätekuljetus
l2 viestintäteollisuus
l2 kestokulutushyödyke
1002 needs to be added manually:  kestohyödyke  -  ke

1414 needs to be added manually:  vesitilasto  -  vesitilastot  -  vesitilastot
1415 needs to be added manually:  valkaisumenetelmä  -  valkaisumenetelmä  -  valkaisumenetelmä
l2 ympäristöyritys
1417 needs to be added manually:  ympäristösodankäynti  -  ympäristösodankäynti  -  ympäristösodankäynti
l2 palkkapolitiikka
l2 luonnonka#asu
1431 needs to be added manually:  maalaustoiminta  -  maalaustoiminta  -  maalaustoiminta
1435 needs to be added manually:  kuuloelimistö  -  kuuloelimistö  -  kuuloelimistö
l2 ympäristötilasto
1442 needs to be added manually:  eläintuote  -  eläintuote  -  eläintuote
1444 needs to be added manually:  jokivara  -  jokivarat  -  jokivarat
l2 asumistiheys
l2 biotooppi
l2 asuntoasia
l2 valuuttapolitiikka
l2 nurmialue
l2 teollisuuskaatopaikka
l2 kasvillisuus#peite
l2 kuulonsuojaus
1458 needs to be added manually:  lainkäyttöviranomainen  -  lainkäyttöviranomainen  -  lainkäyttöviranomainen
1463 needs to be added manually:  lämmönakkumulaatio  -  lämmönakkumul

1753 needs to be added manually:  vesitie#alue  -  vesitiealue  -  yleinen vesitiealue
l2 turvallisuusnormi
1758 needs to be added manually:  kaupunkitoimintaohjelma  -  kaupunkitoimintaohjelma  -  kaupunkitoimintaohjelma
l2 ympäristötilin#pito
1762 needs to be added manually:  jätevesilainsäädäntö  -  jätevesilainsäädäntö  -  jätevesilainsäädäntö
l2 nettovähennys
l2 ympäristöarviointi
1766 needs to be added manually:  kasvinsuojelutuote  -  kasvinsuojelutuote  -  kasvinsuojelutuote
1768 needs to be added manually:  rajakuormitus  -  rajakuormitus  -  rajakuormitus kriittinen kuormitus
1775 needs to be added manually:  betonituoteteollisuus  -  betonituoteteollisuus  -  betonituoteteollisuus
1777 needs to be added manually:  maamalli  -  maamalli  -  numeerinen maamalli
1778 needs to be added manually:  ympäristöterveysvaara  -  ympäristöterveysvaara  -  ympäristöterveysvaara
1790 needs to be added manually:  maankäyttöjärjestelmä  -  maankäyttöjärjestelmä  -  maankäyttöjärjestelmä
179

...

3406 needs to be added manually:  poistokaasulaite  -  poistokaasulaite  -  poistokaasulaite
l2 käyttäytymistiee
3410 needs to be added manually:  saastumisriski  -  saastumisriski  -  saastumisriski
l2 kuluttajavalistus
3415 needs to be added manually:  suolanpoistolaitos  -  suolanpoistolaitos  -  suolanpoistolaitos
l2 saass
3420 needs to be added manually:  mereenpurkupaikka  -  mereenpurkupaikka  -  mereenpurkupaikka
3421 needs to be added manually:  kasvitekstiilikuida  -  kasvitekstiilikuitu  -  kasvitekstiilikuitu
l2 saass
l2 kivi#työ
l2 erikoiskemikaali
3427 needs to be added manually:  tervahiekka  -  tervahiekka  -  tervahiekka
l2 hiekanotto
w3 kloroota
l2 tartutus
3434 needs to be added manually:  aktiiviliete  -  aktiiviliete  -  aktivoitu liete aktiiviliete
l2 höytyä
l2 saass
3443 needs to be added manually:  suolasuo  -  suolasuo  -  suolasuo
3449 needs to be added manually:  syövytyslaitos  -  syövytyslaitos  -  syövytyslaitos peittauslaitos
3449 needs to be added manual

3779 needs to be added manually:  savusumuvaroitus  -  savusumuvaroitus  -  savusumuvaroitus
3780 needs to be added manually:  immissionvalvonta  -  immissionvalvonta  -  immissionvalvonta
3784 needs to be added manually:  rantarius  -  rantariutta  -  rantariutta valliriutta
3785 needs to be added manually:  raskasmetallikuormitus  -  raskasmetallikuormitus  -  raskasmetallikuormitus
l2 jätelata
3790 needs to be added manually:  tulvasuojeluallas  -  tulvasuojeluallas  -  tulvasuojeluallas
l2 hyvitysjärjestelmä
l2 kompostoitaa
3797 needs to be added manually:  jäteliejukomposti  -  jäteliejukomposti  -  jäteliejukomposti
w1 tuntoella
l2 fotokemiallinen
l2 sopeutumis#aika
3804 needs to be added manually:  jätekaasu  -  jätekaasun  -  biologinen jätekaasun puhdistus
l2 romumetalli
l2 uusiopaperi
l2 zoonoo
l2 idätys
l2 nivelmato
l2 polymerointi
l2 polymeroituminen
l2 rikki#happo
l2 fosforiyhdiste
3827 needs to be added manually:  melupäästö#maksu  -  melupäästömaksu  -  melupäästömaksu
l

w2 hallintotuomioistua
l2 magneetti#nauha
4168 needs to be added manually:  saastelähs  -  saastelähteen  -  saastelähteen toteaminen
4169 needs to be added manually:  jätekipsi  -  jätekipsi  -  jätekipsi
4171 needs to be added manually:  jälkitilaus  -  jälkitilaus  -  jälkitilaus
l2 ionosfääri
l2 ilmakuvaus
l2 polysyklinen
l2 hiilivedä
w3 kivihiillä
l2 vetysulfidi
l2 tullitariffa
4192 needs to be added manually:  tullitaksaa  -  tullitaksa  -  tullitariffi tullitaksa
4193 needs to be added manually:  vaahdotusaine  -  vaahdotusaine  -  vaahdotusaine
4194 needs to be added manually:  kasvisosiologia  -  kasvisosiologia  -  kasvisosiologia
4195 needs to be added manually:  kartanpiirustus  -  kartanpiirustus  -  kartanpiirustus
l2 saass
4202 needs to be added manually:  paleoekologia  -  paleoekologia  -  paleoekologia
l2 hätämajoitus
4205 needs to be added manually:  assimilaatio#kyky  -  assimilaatiokyky  -  jätteen assimilaatiokyky
4206 needs to be added manually:  saasteindikaatto

l2 tetraklooriee
4541 needs to be added manually:  perkloorietyle  -  perkloorietyleeni  -  tetrakloorieteeni perkloorietyleeni
4545 needs to be added manually:  aktinidi  -  aktinidi  -  aktinidi
4547 needs to be added manually:  immissiokuormitus  -  immissiokuormitus  -  immissiokuormitus
4549 needs to be added manually:  suolapahku  -  suolapahku  -  suolapahku
4550 needs to be added manually:  lietteenlaskeutusallas  -  lietteenlaskeutusallas  -  lietteenlaskeutusallas
l2 jätteenpolton
4553 needs to be added manually:  emulgoiminen  -  emulgoiminen  -  emulgoiminen
l2 freo
l2 analyytä
4564 needs to be added manually:  kloorivetyhappo  -  kloorivetyhappo  -  kloorivetyhappo suolahappo
l2 saha#jauho
4570 needs to be added manually:  ilki#teko  -  ilkiteko  -  ilkiteko laiminlyönti
l2 viadukti
!!!!! error - not equal length:  [] ['']
4581 needs to be added manually:  kodifioiminen  -  kodifioiminen  -  kodifioiminen
4584 needs to be added manually:  kasvitautioppi  -  kasvitautioppi 

l2 geotekniikka
4998 needs to be added manually:  jäätiköntutkimus  -  jäätiköntutkimus  -  jäätiköntutkimus jäätikkötiede
l2 jäätikkö#tiede
5002 needs to be added manually:  karsinogeenisyyskoe  -  karsinogeenisyyskoe  -  karsinogeenisyyskoe
5003 needs to be added manually:  jätteenpolttou  -  jätteenpolttouuni  -  jätteenpolttouuni
5008 needs to be added manually:  korologia  -  korologia  -  korologia
5010 needs to be added manually:  teratogeneesi  -  teratogeneesi  -  teratogeneesi
l2 rikinpoisto
5020 needs to be added manually:  valekuristustauti  -  valekuristustauti  -  valekuristustauti
5021 needs to be added manually:  säiliötor  -  säiliötorni  -  siilo säiliötorni
5022 needs to be added manually:  viskoosisuus  -  viskoosisuus  -  viskoosisuus sitkaus sitkaisuus sitko
5022 needs to be added manually:  sitkaus  -  sitkaus  -  viskoosisuus sitkaus sitkaisuus sitko
5022 needs to be added manually:  sitkaisuus  -  sitkaisuus  -  viskoosisuus sitkaus sitkaisuus sitko
l2 sieni#ti

In [19]:
# for manual correcting of words
pprint(words_to_manual_add)

{'aerosolipullo': 'aerosolipullo spraypurkki',
 'agroekosysteemi': 'agroekosysteemi',
 'agronominen': 'agronominen lisäaine',
 'aikabudjetti': 'aikabudjetti',
 'aineenkoestus': 'aineenkoestus',
 'ainejäämä': 'torjunta ainejäämä',
 'ainemenekki': 'ainemenekki',
 'ainevalvonta': 'torjunta ainevalvonnan normi',
 'aktiiviliete': 'aktivoitu liete aktiiviliete',
 'aktinidi': 'aktinidi',
 'alatili': 'alatili',
 'alisyklinen': 'alisyklinen yhdiste',
 'alkali#maa': 'emäksinen maa alkalimaa',
 'alkyyliyhdiste': 'alkyyliyhdiste',
 'altistuma': 'altistuma riskin jäämä',
 'aluelainsäädäntö': 'aluelainsäädäntö',
 'alumiinisäiliö': 'alumiinisäiliö',
 'amalgaama': 'amalgaama',
 'ammoniakkiutuminen': 'ammonifikaatio ammoniakkiutuminen',
 'analyysiohjelma': 'analyysiohjelma',
 'anomusoikeus': 'anomusoikeus',
 'asbestisementti': 'asbestisementti',
 'assimilaatio#kyky': 'jätteen assimilaatiokyky',
 'asuinseudulta': 'asuinseudultaan siirtymään joutunut henkilö',
 'asukasjakauma': 'asukasjakauma',
 'asumisj

In [37]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "Etelänapamanner"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

Etelänapamanner


In [21]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [22]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [23]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')
get_vector('διβενζοφουράνιο')

[]

In [24]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [25]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [26]:
print(len(topics),len(topics_nolemma))  

5305 5305


In [27]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

nonexistend word:  uusja  -  uusjako tilusten sijoitus 
Exist nolemma
nonexistend word:  maisemansuunnittelu  -  maisemansuunnittelu 
nonexistend word:  maisemansuunnittelu  -  maisemansuunnittelu 
!!! topic needs to be excluded:  maisemansuunnittelu
nonexistend word:  resurssijohtaminen  -  resurssijohtaminen 
nonexistend word:  resurssijohtaminen  -  resurssijohtaminen 
!!! topic needs to be excluded:  resurssijohtaminen
nonexistend word:  maaeroosio  -  maaeroosio 
nonexistend word:  maaeroosio  -  maaeroosio 
!!! topic needs to be excluded:  maaeroosio
nonexistend word:  ympäristö#muutos  -  ympäristömuutos 
Exist nolemma
nonexistend word:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus 
nonexistend word:  moottoriajoneuvoteollisuus  -  moottoriajoneuvoteollisuus 
!!! topic needs to be excluded:  moottoriajoneuvoteollisuus
nonexistend word:  maaseutusuunnittelu  -  maaseutusuunnittelu 
nonexistend word:  maaseutusuunnittelu  -  maaseutusuunnittelu 
!!! topic needs to be 

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


 yksilöliikenne  -  yksilöliikenne 
!!! topic needs to be excluded:  yksilöliikenne
nonexistend word:  pohjakerrostuma  -  meren pohjakerrostuma merisedimentti 
nonexistend word:  pohjakerrostuma  -  meren pohjakerrostuma merisedimentti 
nonexistend word:  merisedimentti  -  meren pohjakerrostuma merisedimentti 
nonexistend word:  merisedimentti  -  meren pohjakerrostuma merisedimentti 
!!! topic needs to be excluded:  meren pohjakerrostuma, merisedimentti
nonexistend word:  rannikkomeri  -  rannikkomeri rantavesi 
nonexistend word:  rannikkomeri  -  rannikkomeri rantavesi 
!!! topic needs to be excluded:  rannikkomeri, rantavesi
nonexistend word:  suojjelu  -  vesien suojjelu 
nonexistend word:  suojjelu  -  vesien suojjelu 
!!! topic needs to be excluded:  vesien suojjelu
nonexistend word:  maansuojelu  -  maansuojelu 
nonexistend word:  maansuojelu  -  maansuojelu 
!!! topic needs to be excluded:  maansuojelu
nonexistend word:  maansuojelu  -  maansuojelu 
nonexistend word:  maansuo

nonexistend word:  yhteistyöpolitiikka  -  yhteistyöpolitiikka 
nonexistend word:  yhteistyöpolitiikka  -  yhteistyöpolitiikka 
!!! topic needs to be excluded:  yhteistyöpolitiikka
nonexistend word:  metsätalous#tiede  -  metsätaloustiede 
nonexistend word:  metsätaloustiede  -  metsätaloustiede 
!!! topic needs to be excluded:  metsätaloustiede
nonexistend word:  kaksois#talous  -  kaksoistalous 
nonexistend word:  kaksoistalous  -  kaksoistalous 
!!! topic needs to be excluded:  kaksoistalous
nonexistend word:  jäte#tulo  -  jätetulot 
nonexistend word:  jätetulot  -  jätetulot 
!!! topic needs to be excluded:  jätetulot
nonexistend word:  hallintatekniikka  -  hallintatekniikka 
nonexistend word:  hallintatekniikka  -  hallintatekniikka 
!!! topic needs to be excluded:  hallintatekniikka
nonexistend word:  rautatehdasteollisuus  -  rautatehdasteollisuus 
nonexistend word:  rautatehdasteollisuus  -  rautatehdasteollisuus 
!!! topic needs to be excluded:  rautatehdasteollisuus
nonexis

nonexistend word:  ympäristörahasto  -  ympäristörahasto 
nonexistend word:  ympäristörahasto  -  ympäristörahasto 
!!! topic needs to be excluded:  ympäristörahasto
nonexistend word:  rakennuskaavoitus  -  rakennuskaavoitus 
nonexistend word:  rakennuskaavoitus  -  rakennuskaavoitus 
!!! topic needs to be excluded:  rakennuskaavoitus
nonexistend word:  maanjärjestely  -  maanjärjestely 
nonexistend word:  maanjärjestely  -  maanjärjestely 
!!! topic needs to be excluded:  maanjärjestely
nonexistend word:  meriluonnonpuisto  -  meriluonnonpuisto 
nonexistend word:  meriluonnonpuisto  -  meriluonnonpuisto 
!!! topic needs to be excluded:  meriluonnonpuisto
nonexistend word:  hiilitekniikka  -  hiilitekniikka 
nonexistend word:  hiilitekniikka  -  hiilitekniikka 
!!! topic needs to be excluded:  hiilitekniikka
nonexistend word:  kaavoituslainsäädäntö  -  kaavoituslainsäädäntö 
nonexistend word:  kaavoituslainsäädäntö  -  kaavoituslainsäädäntö 
!!! topic needs to be excluded:  kaavoitusla

...

nonexistend word:  tulvasuojeluallas  -  tulvasuojeluallas 
!!! topic needs to be excluded:  tulvasuojeluallas
nonexistend word:  valkaisusavi  -  valkaisusavi valkaisumaa 
nonexistend word:  valkaisusavi  -  valkaisusavi valkaisumaa 
nonexistend word:  valkaisuma  -  valkaisusavi valkaisumaa 
nonexistend word:  valkaisumaa  -  valkaisusavi valkaisumaa 
!!! topic needs to be excluded:  valkaisusavi, valkaisumaa
nonexistend word:  säilytystankki  -  hiilivedyn säilytystankki 
nonexistend word:  säilytystankki  -  hiilivedyn säilytystankki 
!!! topic needs to be excluded:  hiilivedyn säilytystankki
nonexistend word:  äänivallipamaus  -  yliäänipamaus äänivallipamaus 
nonexistend word:  äänivallipamaus  -  yliäänipamaus äänivallipamaus 
!!! topic needs to be excluded:  yliäänipamaus, äänivallipamaus
nonexistend word:  sadelaskeuma  -  sadelaskeuma sadesidonta 
nonexistend word:  sadelaskeuma  -  sadelaskeuma sadesidonta 
nonexistend word:  sadesidoni  -  sadelaskeuma sadesidonta 
nonexist

nonexistend word:  suolakuormitus  -  suolakuormitus 
nonexistend word:  suolakuormitus  -  suolakuormitus 
!!! topic needs to be excluded:  suolakuormitus
nonexistend word:  embryogeneeta  -  embryogeneesi alkionkehitys sikiönkehitys 
nonexistend word:  embryogeneesi  -  embryogeneesi alkionkehitys sikiönkehitys 
!!! topic needs to be excluded:  embryogeneesi, alkionkehitys, sikiönkehitys
nonexistend word:  sähkökinetiikka  -  sähkökinetiikka 
nonexistend word:  sähkökinetiikka  -  sähkökinetiikka 
!!! topic needs to be excluded:  sähkökinetiikka
nonexistend word:  jätehuolto#maksu  -  jätehuoltomaksu 
nonexistend word:  jätehuoltomaksu  -  jätehuoltomaksu 
!!! topic needs to be excluded:  jätehuoltomaksu
nonexistend word:  suurvahinko  -  suurvahinko 
nonexistend word:  suurvahinko  -  suurvahinko 
!!! topic needs to be excluded:  suurvahinko
nonexistend word:  peitausaine  -  siementen peitausaine siemenen puhdistus 
nonexistend word:  peitausaine  -  siementen peitausaine siemenen 

!!!!! error - not equal length:  [] ['']
nonexistend word:  kodifioiminen  -  kodifioiminen 
nonexistend word:  kodifioiminen  -  kodifioiminen 
!!! topic needs to be excluded:  kodifioiminen
nonexistend word:  kasvitautioppi  -  kasvitautioppi 
nonexistend word:  kasvitautioppi  -  kasvitautioppi 
!!! topic needs to be excluded:  kasvitautioppi
nonexistend word:  palautusvaade  -  palautusvaade 
nonexistend word:  palautusvaade  -  palautusvaade 
!!! topic needs to be excluded:  palautusvaade
nonexistend word:  asbestisementti  -  asbestisementti 
nonexistend word:  asbestisementti  -  asbestisementti 
!!! topic needs to be excluded:  asbestisementti
nonexistend word:  joukkoerottelu  -  joukkoerottelu 
nonexistend word:  joukkoerottelu  -  joukkoerottelu 
!!! topic needs to be excluded:  joukkoerottelu
nonexistend word:  naftaleenikie  -  naftaleenikiteet 
nonexistend word:  naftaleenikiteet  -  naftaleenikiteet 
!!! topic needs to be excluded:  naftaleenikiteet
nonexistend word:  hi

nonexistend word:  bioklimatologia  -  bioklimatologia 
!!! topic needs to be excluded:  bioklimatologia
nonexistend word:  teratogeneesitutkimus  -  teratogeneesitutkimus 
nonexistend word:  teratogeneesitutkimus  -  teratogeneesitutkimus 
!!! topic needs to be excluded:  teratogeneesitutkimus
nonexistend word:  bioluminointi  -  bioluminointi elävän organismin fosforoiminen 
nonexistend word:  bioluminointi  -  bioluminointi elävän organismin fosforoiminen 
nonexistend word:  fosforoiminen  -  bioluminointi elävän organismin fosforoiminen 
nonexistend word:  fosforoiminen  -  bioluminointi elävän organismin fosforoiminen 
!!! topic needs to be excluded:  bioluminointi, elävän organismin fosforoiminen
nonexistend word:  eliöntorjunta  -  eliöntorjunta aine 
nonexistend word:  eliöntorjunta  -  eliöntorjunta aine 
!!! topic needs to be excluded:  eliöntorjunta-aine
nonexistend word:  reittää  -  köysi viiva linja rivi raja reitti ala 
Exist nolemma
nonexistend word:  palautusasti  -  p

nonexistend word:  olfaktometria  -  olfaktometria 
!!! topic needs to be excluded:  olfaktometria
nonexistend word:  ituplasma  -  ituplasma 
nonexistend word:  ituplasma  -  ituplasma 
!!! topic needs to be excluded:  ituplasma
nonexistend word:  väliarvolasku  -  interpolaatio väliarvolasku 
nonexistend word:  väliarvolasku  -  interpolaatio väliarvolasku 
!!! topic needs to be excluded:  interpolaatio, väliarvolasku
nonexistend word:  klooriee  -  kloorieteeni 
nonexistend word:  kloorieteeni  -  kloorieteeni 
!!! topic needs to be excluded:  kloorieteeni
nonexistend word:  nitrosamiini  -  nitrosamiini 
nonexistend word:  nitrosamiini  -  nitrosamiini 
!!! topic needs to be excluded:  nitrosamiini
nonexistend word:  gasoholla  -  gasoholi polttoaineseos jossa bensiiniin on lisätty alkoholia 
nonexistend word:  gasoholi  -  gasoholi polttoaineseos jossa bensiiniin on lisätty alkoholia 
!!! topic needs to be excluded:  gasoholi, polttoaineseos, jossa bensiiniin on lisätty alkoholia


In [28]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  906
['maisemansuunnittelu',
 'resurssijohtaminen',
 'maaeroosio',
 'moottoriajoneuvoteollisuus',
 'maaseutusuunnittelu',
 'ruuanjalostusteollisuus',
 'luonnonsuojelupolitiikka',
 'maatalouden tuotantokeinot',
 'ympäristönsuojelusääntö',
 'kesantomaa',
 'maankäyttöjärjestelmä',
 'rannikkoympäristö',
 'maaperän suolautuminene',
 'vesiekologia',
 'meriekologia',
 'maisemanhoito, maisemansuunnittelu',
 'maisemaekologia',
 'maatalousbiotekniikka',
 'maaympäristö',
 'luottoapu',
 'metsätuotanto',
 'vihannesten vilejly',
 'maatalouslainsäädäntö',
 'maatalousvarasto',
 'maataloustiede, maanviljelystaloustiede',
 'ravintotekniikka',
 'maatalousekologia',
 'kehityssuunnittelu',
 'maatalousvälineistö',
 'teollisuuslainsäädäntö',
 'rantaeroosio',
 'lannantuotanto',
 'maaperäkivennäistiede',
 'tihkuvesi',
 'maaseutuarkkitehtuuri',
 'valtakunnansuunnittelu',
 'maannostuminen',
 'yksilöliikenne',
 'meren pohjakerrostuma, merisedimentti',
 'rannikkomeri, rantavesi',
 'vesien suojjelu

In [29]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

paikallinen kehitys  ---  paikallinen_kehitys
maaseuturakentaminen  ---  maaseuturakentaminen
vesitalous  ---  vesi#talous
koulutustaso  ---  koulutus#taso
yleissivistävä koulutus  ---  yleissivistävä_koulutus
rantapenger  ---  rantapenger
yhteisöoikeus  ---  yhteisöoikeus
kulttuuriperintö  ---  kulttuuri#perintö
viljelysmaa, maatalousmaa  ---  viljelys#maa_maa#talous#maa
maatalousmaisema  ---  maatalousmaisema
maataloustuotanto  ---  maataloustuotanto
taloudellinen kehitys  ---  taloudellinen_kehitys
ravintoketju  ---  ravinto#ketju
elintarviketeollisuus  ---  elintarviketeollisuus
ruuan laatu  ---  ruoka_laatu
globaali lämpeneminen  ---  globaali_lämpeneminen
kasvihuoneilmiö  ---  kasvihuoneilmiö
elinkeinoelämän kehitys  ---  elin#keino#elämä_kehitys
työmarkkinat  ---  työ#markkinat
pääsy maa-alueelle  ---  päätä_maa_alue
uusjako, tilusten sijoitus  ---  uusja_tilus_sijoitus
maankäytön suunnittelu  ---  maankäyttö_suunnittelu
maisema-arkkitehtuuri  ---  maisema_arkkitehtuuri
=== EXCL

yhteisön säädös  ---  yhteisö_säädös
maatalousrakennus  ---  maatalousrakennus
teollisuusrakennus  ---  teollisuusrakennus
ilmasto  ---  ilmasto
kosmeettinen teollisuus  ---  kosmeettinen_teollisuus
karjanhoito  ---  karjan#hoito
voimavarojen käyttö  ---  voima#vara_käyttö
veden käyttö  ---  vesi_käyttö
nurmi  ---  nurmi
terästeollisuus  ---  terästeollisuus
sähköteollisuus  ---  sähköteollisuus
vesistöalueen suunnitelmallinen käyttö  ---  vesistöalue_suunnitelmallinen_käyttö
ekosysteemin rappeutuminen  ---  ekosysteemi_rappeutuminen
vaatetusteollisuus  ---  vaatetusteollisuus
keramiikkateollisuus  ---  keramiikkateollisuus
markkinatalous  ---  markkina#talous
ympäristön hoito  ---  ympäristö_hoito
ohjelmiston kehitys  ---  ohjelmisto_kehitys
=== EXCLUDE  maatalousvarasto
maanviljelys  ---  maanviljelys
maataloustuote  ---  maataloustuote
ilmaston muutos  ---  ilmasto_muutos
=== EXCLUDE  maataloustiede, maanviljelystaloustiede
maankäyttö  ---  maankäyttö
teollinen suunnittelu  ---  teo

=== EXCLUDE  ravintotiede
eläinekologia  ---  eläinekologia
geofysikaalinen ympäristö  ---  geofysikaalinen_ympäristö
jokivesi  ---  jokivesi
veden säästäminen  ---  vesi_säästäminen
julkinen rahoitus  ---  julkinen_rahoitus
vuokratalo  ---  vuokra#talo
maan viljeleminen  ---  maa_viljeleminen
jalostus  ---  jalostus
=== EXCLUDE  mikroilmastollinen vaikutus
yhteinen hallinta  ---  yhteinen_hallinta
maaperän funktio  ---  maa#perä_funktio
maanviljelys ja karjanhoito samalla tilalla  ---  maanviljelys_karjan#hoito_tila
kaupungin viheralue  ---  kaupunki_viher#alue
raakavesi  ---  raakavesi
luonnonmetsä  ---  luonnonmetsä
kunnallistaso  ---  kunnallis#taso
mineraaliteollisuus  ---  mineraaliteollisuus
hyvä hallinta  ---  hallinta
=== EXCLUDE  liittymätie, sisääntulotie
yksityisoikeus, siviilioikeus  ---  yksityisoikeus_siviilioikeus
julkisoikeus, julkinen oikeus  ---  julkisoikeus_julkinen_oikeus
=== EXCLUDE  maaperäkemia
luonnontieteet  ---  luonnontiee
=== EXCLUDE  vesitieteet
asukastih

=== EXCLUDE  maakirjarekisteri, lainhuutorekisteri
=== EXCLUDE  työlääketiede
veden kovuus  ---  vesi_kovuus
fyysinen maantiede, luonnonmaantiede  ---  fyysinen_maan#tiede_luonnonmaan#tiede
merikalastus  ---  merikalastus
vesivoima  ---  vesi#voima
rakennuslupa  ---  rakennus#lupa
kaupunki  ---  kaupunki
kalastuslaki  ---  kalastus#laki
=== EXCLUDE  maisemankulutus
pohjavesi  ---  pohjavesi
luonnontieteet  ---  luonnontiee
lait  ---  laki
kaupungin ekosysteemi  ---  kaupunki_ekosysteemi
varastonhallinta  ---  varastonhallinta
tukipolitiikka  ---  tukipolitiikka
kehitysapu  ---  kehitysapu
rahasto  ---  rahasto
riskinhallinta  ---  riskinhallinta
hiilimonoksidi  ---  hiili#monoksidi
maalla tapahtuva interventio  ---  maa_tapahtua_interventio
teuraseläin  ---  teuraseläin
eläinten teurastus  ---  eläin_teurastus
=== EXCLUDE  asukasjakauma
eläintauti  ---  eläintauti
maankäyttö  ---  maankäyttö
vedet  ---  vesi
ammattikunta  ---  ammatti#kunta
alueellinen liikkuvuus  ---  alueellinen_liik

äänitaso  ---  ääni#taso
orgaaninen aine  ---  orgaaninen_aine
=== EXCLUDE  asutusohjelma
asuntovaunu, matkailuperävaunu  ---  asuntovaunu_matkailuperävaunu
lannoitteita koskeva laki  ---  lannoitte_koskea_laki
laadunvalvonta  ---  laadunvalvon
aavikkoilmasto  ---  aavikkoilmasto
teollisuusliitto  ---  teollisuusliitto
luonnonvarainen eläin  ---  luonnonvarainen_eläin
=== EXCLUDE  maaprofiili, maannosprofiili
suojeltu maisema  ---  suojella_maisema
sosiaalipalvelu  ---  sosiaalipalvelu
terveyspalvelu  ---  terveyspalvelu
Karibian meri  ---  karibia_meri
palvelualue  ---  palvelu#alue
julkinen kuuleminen  ---  julkinen_kuuleminen
lääketiede  ---  lääke#tiede
sosiaalitieteet  ---  sosiaalitiee
kotimainen teknologia  ---  koti#mainen_teknologia
valtion velka  ---  valtio_#velka
valmistalo, elementtirakennus  ---  valmis#talo_elementtirakennus
kaupunkiuudistus  ---  kaupunkiuudistus
sosiaalinen indikaattori  ---  sosiaalinen_indikaattori
=== EXCLUDE  talousmaakunta
maailmanlaajuinen malli 

...

jätteiden käsittely, jätehuolto  ---  jäte_käsittely_jätehuolto
puu  ---  puu
lampi  ---  lampi
korroosionestoaine  ---  korroosionestoaine
nitraatti  ---  nitraatti
melusaaste  ---  melusa#aste
kulottunut ruoho  ---  kulottua_ruoho
radioaktiivisuus  ---  radioaktiivisuus
maito  ---  maito
vakuutus  ---  vakuutus
kaasu-, bensiinimoottori  ---  kaasua_bensiinimoottori
asennusten valvonta  ---  asennus_valvonta
sosioekonominen tekijä  ---  sosioekonominen_tekijä
fosfaatin korvaaminen  ---  fosfaatti_korvaaminen
saasteen lähde  ---  saass_lähde
merivirtaukset  ---  merivirtaus
=== EXCLUDE  ylimenopaikka
biologinen typensidonta  ---  biologinen_typensidonta
välineistö, varustaminen tarvittavilla kojeilla  ---  välineistö_varustaminen_tarvittava_kojei
sintraus  ---  sintraus
nälänhätä  ---  nälänhätä
tuomioistuin  ---  tuomio#istuin
sähkötekniset varusteet  ---  sähkö#tekninen_varuste
vapauttaminen, vapautuminen  ---  vapauttaminen_vapautuminen
=== EXCLUDE  dehydratoitu liete
jätteen tila  

nesteytetty kaasu  ---  nesteyttää_kaasu
meren lahti  ---  meri_lahti
=== EXCLUDE  atratsiini
sieni  ---  si
jodi  ---  jodi
dieselpolttoaine  ---  dieselpolttoaine
kirkko  ---  kirkko
=== EXCLUDE  renaturaatio
suoja hätätilanteen varalta  ---  suoja_hätätilans_varalta
vitamiini  ---  vitamiini
siivoaminen  ---  siivoaminen
keuhkosairaus  ---  keuhkosairaus
=== EXCLUDE  jätetasapaino
harvinainen laji  ---  harvinainen_laji
savukaasu, palokaasu, poistokaasu  ---  savukaasua_paloka#asu_poistoka#asu
tautia aiheuttava organismi  ---  tauti_aiheuttaa_organismi
kaatopaikka, täyttömaa  ---  kaato#paikka_täyttöma
väestönlaskenta  ---  väestönlasken
palkkio, maksu  ---  palkkio_maksu
=== EXCLUDE  aerosolipullo, spraypurkki
merenpohjan hyväksikäyttö  ---  merenpohja_hyväksi#käyttö
matka  ---  matka
tilintarkastus  ---  tilintarkastus
muutoksenhakumenettely  ---  muutoksenhakumenettely
=== EXCLUDE  virtamittaus
melueste  ---  melueste
hyväksyminen, vahvistaminen  ---  hyväksyminen_vahvistaminen
i

hallitseva  ---  hallitseva
hälytys  ---  hälytys
=== EXCLUDE  puukuorman purkaminen
laskenta  ---  lasken
päätöslauselma  ---  päätös#lauselma
polttomoottori  ---  polttomoottori
epämetallit  ---  epämetalli
laboratoriokoe  ---  laboratoriokoe
puutarha  ---  puu#tarha
maalikauppa  ---  maali#kauppa
hallituksen virheellinen toimenpide  ---  hallitus_virheellinen_toimen#pide
=== EXCLUDE  jäteveden levittämiskielto
=== EXCLUDE  alumiinisäiliö
laji  ---  laji
paimentolainen  ---  paimentolainen
ruskohiilen louhinta  ---  ruskohiili_louhinta
=== EXCLUDE  silppurijäte
kasvilajin uudelleen tuominen ympäristöön  ---  kasvilakea_tuominen_ympäristö
norsunluu  ---  norsunlua
savusumu  ---  savusumu
hallitus  ---  hallitus
asiakirjojen luettelointi  ---  asia#kirja_luettelointi
auringonpaisteen kesto  ---  auringonpaiss_kesto
tilastotiedot  ---  tilastotieto
=== EXCLUDE  atomiräjähdys
tasainen lähde  ---  tasainen_lähde
ruo'ot, kaislat  ---  ruo_ko_kaisla
myrkyttömäksi tekeminen, myrkyn poisto  -

liukenematon aine  ---  liukenematon_aine
säteily  ---  säteily
ministeriö  ---  ministeriö
veden virtausta koskeva tutkimus ja mittaus  ---  vesi_virtaus_koskea_tutkimus_mittaus
Oseania  ---  oseani
ympäristöön rikastuva tekijä  ---  ympäristö_rikastua_tekijä
sudenkorennot  ---  sudenkorento
=== EXCLUDE  tietoväline, tiedonsiirtoväline
suljettu alue  ---  sulkea_alue
hieno  ---  hieno
riita-asia, käräjöinti  ---  riita_käräjöinti
hajanainen lähde  ---  hajanainen_lähde
tuhka  ---  tuhka
kromi  ---  kromi
tina  ---  ti
merkitsin  ---  merkitsi
=== EXCLUDE  halogenoitu difenyyli
aseistautuminen, varustelu  ---  aseistautuminen_varustelu
helposti syttyvä aine  ---  syttyä_aine
nelitahtimoottori  ---  nelitahtimoottori
kaksitahtimoottori  ---  kaksitahtimoottori
potaska  ---  potaska
nikkeli  ---  nikkeli
päiväys  ---  päiväys
alkio  ---  alkio
=== EXCLUDE  kuvanmittaus
kasvitiede  ---  kasvi#tiede
=== EXCLUDE  itsepuhdistuminen
kansallistaminen  ---  kansallistaminen
asettaa näytteille  

=== EXCLUDE  esikäsittely, esioikeudenkäynti
ääni-immissio  ---  ääni_immissio
=== EXCLUDE  linnunhoito
(tieteellinen) nimistö, sanaluettelo  ---  tieteellinen_nimistö_sana#luettelo
=== EXCLUDE  hivenaineet, hivenalkuaineet
epämuodostuma  ---  epämuodostuma
metalliromun kaatopaikka  ---  metalliromu_kaato#paikka
metalliseos  ---  metalliseos
puolustus  ---  puolustus
lentotuhka  ---  lentotuhka
=== EXCLUDE  kidetiede
plutonium  ---  plutonium
ympäristölle vieras aine  ---  ympäristö_vieras_aine
mukavuuslippu  ---  mukavuuslippu
immuniteetti  ---  immuniteetti
bakteereita tappava aine  ---  bakteeri_tappaa_aine
kuolio  ---  kuolio
mutaation läpikäyneiden mikro-organismien päästö  ---  mutaatio_läpikäya_mikro_organismi_päästö
eetteri  ---  eetteri
=== EXCLUDE  lahdeke, luonnon valkama
kerosiini, paloöljy  ---  kerosiini_paloöljy
rinnakkaispoltto  ---  rinnakkaispoltto
suisto  ---  suisto
annos, annostus  ---  annos_annostus
pentakloorifenoli  ---  pentakloorifenoli
lajittelu syntypaikall

In [30]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  paikallinen_kehitys
--- only one word, not a new topic> maaseuturakentaminen
--- only one word, not a new topic> vesi#talous
--- only one word, not a new topic> koulutus#taso
append  yleissivistävä_koulutus

NOT in vocabulary:  rantapenger rantapenger
append  rantapenger
--- only one word, not a new topic> rantapenger

NOT in vocabulary:  yhteisöoikeus yhteisöoikeus
append  yhteisöoikeus
--- only one word, not a new topic> yhteisöoikeus
--- only one word, not a new topic> kulttuuri#perintö
append  viljelys#maa_maa#talous#maa

NOT in vocabulary:  maatalousmaisema maatalousmaisema
append  maatalousmaisema
--- only one word, not a new topic> maatalousmaisema
--- only one word, not a new topic> maataloustuotanto
append  taloudellinen_kehitys
--- only one word, not a new topic> ravinto#ketju
--- only one word, not a new topic> elintarviketeollisuus
append  ruoka_laatu
append  globaali_lämpeneminen
--- only one word, not a new topic> kasvihuoneilmiö
append  elin#keino#elämä_kehitys
-

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


--- only one word, not a new topic> huonekaluteollisuus

NOT in vocabulary:  lujitus maa_lujitus_maa_stabilointi
append  lujitus
append  maa_lujitus_maa_stabilointi
append  taloudellinen_tuki
append  varain#hoito_taloushallinto

NOT in vocabulary:  kulutusta#vara kulutusta#vara
append  kulutusta#vara
--- only one word, not a new topic> kulutusta#vara
append  vettyä_maa
--- only one word, not a new topic> kulttuuripolitiikka
--- only one word, not a new topic> hotelli#ala
append  vesi_tarve
append  talous_ala
append  kriittinen_taso
append  metsä_ekosysteemi
--- only one word, not a new topic> rannikko#alue
--- only one word, not a new topic> maa#talous
append  vesi_hinnoittelu

NOT in vocabulary:  maakuljetus maakuljetus
append  maakuljetus
--- only one word, not a new topic> maakuljetus
append  julkinen_liikenne
append  kuiva_maa
append  luonnon#vara_hyväksi#käyttö
append  vesi_laatu
--- only one word, not a new topic> aikuiskoulutus
append  varo_suojelu

NOT in vocabulary:  laatua ve

NOT in vocabulary:  eliömaantieteellinen eliömaantieteellinen_alue
append  eliömaantieteellinen
append  eliömaantieteellinen_alue
--- only one word, not a new topic> kulkulaitos

NOT in vocabulary:  rakennustontti rakennusa_rakennustontti
append  rakennustontti
append  rakennusa_rakennustontti
--- only one word, not a new topic> rahamarkkina
append  yhteis#kunnallinen_prosessi
append  ympäristön#suojelu_taso
--- only one word, not a new topic> energiatekniikka
append  elollinen_luonto_monimuotoisuus
append  rahallinen_korvaus
append  kaupunki_puhtaana#pito
append  vesi_uudelleenkäyttö
--- only one word, not a new topic> ase#voima
append  maa#talous_tuhoeläin
append  luonnonvarainen_alue
--- only one word, not a new topic> liiketoiminta
append  yhteis#kunnallinen_tilanne

NOT in vocabulary:  sokeriteollisuus sokeriteollisuus
append  sokeriteollisuus
--- only one word, not a new topic> sokeriteollisuus
--- only one word, not a new topic> laivanrakennus
append  hallinto_oikeus
append  dem

append  luonnononnettomuus
--- only one word, not a new topic> luonnononnettomuus
append  asutus_taajama
append  eläin_käyttäytyminen
--- only one word, not a new topic> markkinatutkimus

NOT in vocabulary:  kauppapuutarha kauppapuutarha_#pito
append  kauppapuutarha

NOT in vocabulary:  #pito kauppapuutarha_#pito
append  #pito
append  kauppapuutarha_#pito
--- only one word, not a new topic> kasvisto
append  laki_täytäntöönpano
--- only one word, not a new topic> nuoriso#työ
--- only one word, not a new topic> voimaanpano#laki
append  huoli_ympäristö
append  julkinen_laitos

NOT in vocabulary:  hiiliteollisuus hiiliteollisuus
append  hiiliteollisuus
--- only one word, not a new topic> hiiliteollisuus
--- only one word, not a new topic> yhteis#kunta
append  sosiaalinen_analyysi
--- only one word, not a new topic> kulttuuripalvelu
--- only one word, not a new topic> alue
append  päästö_vesi

NOT in vocabulary:  säästösuunnitelma taloussuunnitelma_säästösuunnitelma
append  säästösuunnitelm

...

--- only one word, not a new topic> kaivosteollisuus
--- only one word, not a new topic> tieto#keskus
--- only one word, not a new topic> organismi
--- only one word, not a new topic> hiiva
--- only one word, not a new topic> kirjan#pito
append  geneettisesti_muuttaa_organismi
append  kiinteä_hiukkanen

NOT in vocabulary:  inervita inervita_jäte
append  inervita
append  inervita_jäte
--- only one word, not a new topic> kerrostuminen
--- only one word, not a new topic> kansalainen

NOT in vocabulary:  saass saass_kulkeutuminen
append  saass_kulkeutuminen
append  saastuttaa_aine_saase
append  ympäristö_kertyä_epäpuhtaus
--- only one word, not a new topic> avaruus
append  laajuus_määrääminen
append  jäte_vähentäminen
append  kevyt_neste_erottaa
--- only one word, not a new topic> sana#kirja

NOT in vocabulary:  jatkuvakäyttöinen jatkuvakäyttöinen_säiliö
append  jatkuvakäyttöinen
append  jatkuvakäyttöinen_säiliö
--- only one word, not a new topic> edustus
--- only one word, not a new topic

append  pintakasvillisuus
append  pintakasvillisuus_poisto
!!!!! error - not equal length:  [] ['']
--- only one word, not a new topic> nukleiinihappo

NOT in vocabulary:  varokeino varokeino_varalta
append  varokeino
append  varokeino_varalta
--- only one word, not a new topic> viite#tieto

NOT in vocabulary:  lainsäädäntöeli lainsäädäntöeli
append  lainsäädäntöeli
--- only one word, not a new topic> lainsäädäntöeli
append  vihreä_liike_ekologinen_liike
--- only one word, not a new topic> lämpö#tila
--- only one word, not a new topic> tuonti
--- only one word, not a new topic> farmakokinetiikka
--- only one word, not a new topic> öljynporaus

NOT in vocabulary:  haihtumaton haihtumaton_aine
append  haihtumaton
append  haihtumaton_aine
--- only one word, not a new topic> yhdistys

NOT in vocabulary:  alumiinipitoisuus alumiinipitoisuus
append  alumiinipitoisuus
--- only one word, not a new topic> alumiinipitoisuus

NOT in vocabulary:  teeta teeta_väite
append  teeta
append  teeta_väite

NOT in vocabulary:  sulkuportti sulkuportti
append  sulkuportti
--- only one word, not a new topic> sulkuportti
append  ympäristötietoisuus_ympäristö#liike
--- only one word, not a new topic> puhdistus#aine
append  kuollut_hautaus
--- only one word, not a new topic> vika
--- only one word, not a new topic> tuomio
--- only one word, not a new topic> ydinreaktori
append  vaarallinen_kaato#paikka
--- only one word, not a new topic> huolen#aihe

NOT in vocabulary:  ydinvahingoinen vastuu_ydinvahingoinen
append  ydinvahingoinen
append  vastuu_ydinvahingoinen

NOT in vocabulary:  asbestoo asbestoo
append  asbestoo
--- only one word, not a new topic> asbestoo
append  rannikko_merivartiosto

NOT in vocabulary:  kutupaikka kutupaikka
append  kutupaikka
--- only one word, not a new topic> kutupaikka
--- only one word, not a new topic> alkoholi
--- only one word, not a new topic> valkaisuaine
append  maa#pallo_muodostuminen_liittyä_tekijä
--- only one word, not a new topic> sana#kirja

NOT in voc

NOT in vocabulary:  köytää köytää_viiva_linja_rivi_raja_reittää_ala
append  köytää

NOT in vocabulary:  reittää köytää_viiva_linja_rivi_raja_reittää_ala
append  reittää
append  köytää_viiva_linja_rivi_raja_reittää_ala
--- only one word, not a new topic> uuni
--- only one word, not a new topic> säästö

NOT in vocabulary:  metsästys#lupa metsästys#lupa
append  metsästys#lupa
--- only one word, not a new topic> metsästys#lupa
--- only one word, not a new topic> paleontologia
append  savustus_kaasutus
--- only one word, not a new topic> mielipidetiedustelu
append  piste_kärki
--- only one word, not a new topic> liima

NOT in vocabulary:  immissio ääni_immissio
append  ääni_immissio

NOT in vocabulary:  sana#luettelo tieteellinen_nimistö_sana#luettelo
append  sana#luettelo
append  tieteellinen_nimistö_sana#luettelo
--- only one word, not a new topic> epämuodostuma
append  metalliromu_kaato#paikka
--- only one word, not a new topic> metalliseos
--- only one word, not a new topic> puolustus



append  maa#perä_huonontuminen

NOT in vocabulary:  kaatopaikkadirektiivi kaatopaikkadirektiivi
append  kaatopaikkadirektiivi
--- only one word, not a new topic> kaatopaikkadirektiivi
append  meri#liikenne_poltto#aine
append  yhdyskuntajäts_syntyä

NOT in vocabulary:  yhdyskuntajätevesi yhdyskuntajätevesi
append  yhdyskuntajätevesi
--- only one word, not a new topic> yhdyskuntajätevesi

NOT in vocabulary:  nitraattidirektiivi nitraattidirektiivi
append  nitraattidirektiivi
--- only one word, not a new topic> nitraattidirektiivi
append  eloperäinen_maanparannusaine
append  organisaatio_ympäristöjalanjälki_koskea_menetelmä
--- only one word, not a new topic> otsonikato
append  poliittinen_kehys
append  politiikka_täytäntöönpano
--- only one word, not a new topic> primäärienergia
append  tuote_ympäristöjalanjälki_koskea_menetelmä

NOT in vocabulary:  tuottajuus käyttäjä_tuottajuus
append  tuottajuus
append  käyttäjä_tuottajuus
append  uusiutua_energialähte_koskea_direktiivi
--- only one w

In [32]:
# add topics to model
model.add(new_words,new_vectors)

In [33]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [34]:
len(model.index2word)

157796

In [39]:
# try the model
model.similar_by_vector('liiallinen_lannoittaminen')

[('liiallinen', 0.8435625433921814),
 ('lannoittaminen', 0.8320568203926086),
 ('liiallinen_hyväksi#käyttö', 0.7847802639007568),
 ('liiallisuus', 0.7568169236183167),
 ('tuholais_aiheuttaa_ruoka_pilaantuminen', 0.7261871099472046),
 ('kemiallinen_tuholais_torjunta', 0.7054484486579895),
 ('lisä#aine_päivittäinen_saanti_terveydellinen_vaara', 0.6938666105270386),
 ('ylenmääräinen', 0.6806185841560364),
 ('biologinen_torjunta', 0.6746164560317993),
 ('saastuminen_biologinen_vaikutus', 0.674130916595459)]

In [40]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 paikallinen_kehitys
0 [('taloudellinen_kehitys', 0.8796688914299011), ('historiallinen_kehitys', 0.876581072807312), ('sosiaalinen_kehitys', 0.8748709559440613), ('kehitys', 0.8701858520507812), ('kulttuuri_kehitys', 0.8560131788253784), ('ympäristö_kehitys', 0.8559619188308716), ('ekologinen_kehitys', 0.8475098609924316), ('elin#keino#elämä_kehitys', 0.8325319886207581), ('kestävä_kehitys_tavoite', 0.8262125849723816), ('ohjelmisto_kehitys', 0.8191323280334473)]

-----
 maaseuturakentaminen
1 [('maaseuturahoitus', 0.6721558570861816), ('hajarakentaminen', 0.6029298305511475), ('aluerakentaminen', 0.5936902761459351), ('puurakentaminen', 0.5914800763130188), ('pientalorakentaminen', 0.5743743181228638), ('tuulivoimarakentaminen', 0.56592857837677), ('palvelurakentaminen', 0.5575456023216248), ('ympäristörakentaminen', 0.5517005324363708), ('rakentaminen', 0.5515429377555847), ('maa#seutu', 0.5498995780944824)]

-----
 vesi#talous
2 [('vesitalous', 0.7962394952774048), ('vesitas

24 [('maankäyttö_suunnittelu', 0.9694346189498901), ('kaavoitus', 0.932364284992218), ('maankäyttö', 0.9165095090866089), ('yleiskaavoitus', 0.8512250185012817), ('kaavoituknen', 0.8399124145507812), ('kaavoitus#työ', 0.8375333547592163), ('maankäytönsuunnittelu', 0.8321706056594849), ('maankäytö', 0.8315062522888184), ('asemakaavoitus', 0.8247443437576294), ('kaavoitusohjelma', 0.7952760457992554)]

-----
 meriympäristö
25 [('meriympäristö_seuranta', 0.7873454093933105), ('vesiympäristö', 0.7645580768585205), ('meri_ekosysteemi', 0.7249875068664551), ('meri_eläimistö', 0.710669755935669), ('kaivostoims_meri', 0.7064034938812256), ('meri#vesi_suojelu', 0.705212414264679), ('merialue', 0.7031426429748535), ('meri_saastuminen', 0.7023960947990417), ('merialuei', 0.7022335529327393), ('ympäristö', 0.6949877738952637)]

-----
 luonnonsuojelu
26 [('luonnonsuojelualue', 0.7741366624832153), ('luonnonsuojelujärjestö', 0.7486464381217957), ('luonnon#vara_suojelu', 0.7450330257415771), ('luonno

46 [('makea_vesi_organismi', 0.9571949243545532), ('makea_vesi_saastuminen', 0.9473932385444641), ('makea_vesi_biologia', 0.9467605352401733), ('makea_vesi_pilaantuminen', 0.9433822631835938), ('makea_vesi_suojelu', 0.9277530908584595), ('vesi_ekosysteemi', 0.9271630048751831), ('makea_vesi_seuranta', 0.925407350063324), ('suolainen_vesi', 0.9208741188049316), ('makea_suolaton_vesi', 0.9048411250114441), ('vesi_suolaisuus', 0.9008100032806396)]

-----
 vesi_ekosysteemi
47 [('makea_vesi_ekosysteemi', 0.9271630048751831), ('vesi', 0.9222490787506104), ('vesi_vuorovaikutus', 0.9083489179611206), ('vesi_saastuminen', 0.8960654139518738), ('vesi_uudelleenkäyttö', 0.8927696347236633), ('vesi_virtaus', 0.8801121115684509), ('vesi_maatalouskäyttö', 0.8789336681365967), ('vaihettumisvyöhyke_vesi', 0.8764654397964478), ('kaupunki_vesi', 0.8708679676055908), ('vesi_kiertokulku_luonno', 0.8693335056304932)]

-----
 vesi#tie
48 [('vesistö_vesi#tie_väylä', 0.8573058247566223), ('vesiti', 0.745229661

69 [('maa_alue_suojelu', 0.917488694190979), ('luonnonvarainen_alue', 0.88107830286026), ('vuoristoalue_suojelu', 0.8756961822509766), ('luonnon#vara_suojelu', 0.8752777576446533), ('luonto#tyyppi_suojelu', 0.8430526256561279), ('suojelu', 0.8430050015449524), ('suojelu#alue_rauhoittaa_alue', 0.8282761573791504), ('eläin_suojelu', 0.8221214413642883), ('eliömaantieteellinen_alue', 0.8188965320587158), ('maa_alue_ekosysteemi', 0.817983865737915)]

-----
 luonnonsuojelujärjestö
70 [('ympäristönsuojelujärjestö', 0.768477201461792), ('lastensuojelujärjestö', 0.7577179670333862), ('luonnonsuojelu', 0.7486464977264404), ('luonnonsuojeluyhdistys', 0.7283951044082642), ('ympäristöjärjestö', 0.7220127582550049), ('luonnon#suojelu#liitto', 0.6931795477867126), ('luonnonsuojeluohjelma', 0.6874432563781738), ('luonnonsuojelualue', 0.6737980246543884), ('ympäristö#järjestö', 0.671500027179718), ('luonnonsuojeluliitto', 0.6638280153274536)]

-----
 kaupunki#alue
71 [('suurkaupunki#alue', 0.854755640

95 [('lisä#aine_päivittäinen_saanti_terveydellinen_vaara', 0.8375183939933777), ('saanti', 0.831143856048584), ('ruoka_laatu', 0.8246186971664429), ('ruoka', 0.7995151281356812), ('ruoka_saastuminen', 0.794216513633728), ('ruoka_säilytys', 0.7835275530815125), ('ruoka_varastointi', 0.7824837565422058), ('kaasu_saanti', 0.7796590328216553), ('runsasproteiininen_ruoka', 0.7673540711402893), ('pää#asiallinen_ravinto_tärkeä_ravinto', 0.7630892395973206)]

-----
 eläin_terveys
96 [('eläin_ravitsemus', 0.9216883778572083), ('eläin', 0.8874942064285278), ('eläin_käyttäytyminen', 0.8836535215377808), ('ihminen_terveys', 0.8751778602600098), ('luonnonvarainen_eläin', 0.8699684143066406), ('eläin_lisääntyminen', 0.8577101230621338), ('eläin_ruokinta', 0.8492965698242188), ('eläin_teurastus', 0.8386462330818176), ('eläin_suojelu', 0.829155445098877), ('eläin_oikeus', 0.8238361477851868)]

-----
 maa
97 [('valtio_maa', 0.9289814829826355), ('maa_alue', 0.887969434261322), ('kuiva_maa', 0.869154214

...

4267 [('puitedirektiivi', 0.7275362014770508), ('päästökauppadirektiivi', 0.714402437210083), ('kaatopaikkadirektiivi', 0.7005561590194702), ('vesipolitiikka_puitedirektiivi', 0.6972614526748657), ('lintudirektiivi', 0.6946154832839966), ('konedirektiivi', 0.6852130889892578), ('energiatehokkuusdirektiivi', 0.6801626682281494), ('direktiivi', 0.6799631118774414), ('energiapalveludirektiivi', 0.6663485765457153), ('kemikaalilainsäädäntö', 0.6529744863510132)]

-----
 eläinjäts_käsittely
4268 [('eläinjäts', 0.8904767036437988), ('kemiallinen_jäte_käsittely', 0.776432991027832), ('radio#aktiivinen_jäte_käsittely', 0.7367147207260132), ('jäte_käsittely_jätehuolto', 0.7338278889656067), ('biologinen_jäte#vesi_käsittely', 0.7327790260314941), ('purkujäts', 0.7270720601081848), ('kierrätys_hallinta_jäte_koskea_laki', 0.723037600517273), ('viljelykasva_käsittely', 0.716367244720459), ('liets_käsittely', 0.7156780958175659), ('kemiallinen_käsittely', 0.7051184773445129)]

-----
 ihminen_muokata

4294 [('haavoittuvuus', 0.8345199823379517), ('ekosysteemipalvelu', 0.8188679218292236), ('ekosysteemi', 0.7463183403015137), ('ekosysteemi_vuorovaikutus', 0.7299563884735107), ('ekosysteemi_tutkimus', 0.7231754660606384), ('ekosysteemipalveluinen', 0.7085709571838379), ('kaupunki_ekosysteemi', 0.671978235244751), ('ekosysteemi_rappeutuminen', 0.6668537855148315), ('metsä_ekosysteemi', 0.6658734083175659), ('luonnon#vara_hallinta', 0.638822615146637)]

-----
 tehokkuus_arviointi
4295 [('ympäristövaikutus_arviointi', 0.8683916330337524), ('arviointi', 0.8602390885353088), ('vaikutus_arviointi', 0.8594055771827698), ('ympäristö_taloudellinen_arviointi', 0.8450981378555298), ('tekniikka_arviointi', 0.8374037742614746), ('tehokkuus_parantaminen', 0.8290325403213501), ('ympäristö_terveysvaikutus_arviointi', 0.8265652060508728), ('kirjallisuus_arviointi', 0.8204774856567383), ('tehokkuus', 0.8178476095199585), ('rahallinen_arviointi', 0.8163880109786987)]

-----
 päästöoikeus
4296 [('päästöo

4323 [('massatase', 0.8563292026519775), ('jäätikkö', 0.8330351114273071), ('jäätikö', 0.7398561239242554), ('mannerjäätikkö', 0.6743516325950623), ('jäätiköi', 0.664466381072998), ('jäätikköjoki', 0.6461663246154785), ('jäätikä', 0.6404570937156677), ('jäätiköityminen', 0.6310835480690002), ('mannerjäätikö', 0.6304545402526855), ('lakijäätikkö', 0.6201050281524658)]

-----
 maa#perä_huonontuminen
4324 [('maa#perä_vaurioituminen', 0.8715626001358032), ('maa#perä_tiivistyminen', 0.8684511184692383), ('maa#perä_saastuminen', 0.8646259307861328), ('maa#perä', 0.83943772315979), ('maa#perä_koostumus', 0.8184604644775391), ('maa#perä_laatu', 0.8158422708511353), ('maa#perä_kehitysprosessi', 0.8119615316390991), ('maa#perä_sieto#kyky', 0.8112244606018066), ('maa#perä_suolaisuus', 0.8017116785049438), ('maa#perä_liukeneminen', 0.7987927794456482)]

-----
 kaatopaikkadirektiivi
4325 [('päästökauppadirektiivi', 0.7413251996040344), ('ilmanlaatudirektiivi', 0.7005561590194702), ('kaato#paikka', 

4351 [('tehokkuus', 0.8485682606697083), ('politiikka_suunnittelu', 0.8380545377731323), ('politiikka', 0.8298026323318481), ('politiikka_yhtenäistäminen', 0.827955961227417), ('politiikka_täytäntöönpano', 0.8044407367706299), ('taloudellis_ekologinen_tehokkuus', 0.8013286590576172), ('politiikka_väline', 0.7870754599571228), ('politiikka_menettely#tapa_ohjelma', 0.7840700745582581), ('tehokkuus_parantaminen', 0.7750875949859619), ('tehokkuus_arviointi', 0.7693675756454468)]

-----
 lämpö#tila_muutos
4352 [('lämpö#tila', 0.8443029522895813), ('ilmasto_muutos', 0.8227882981300354), ('lämpötila', 0.7836090326309204), ('fysikaalinen_muutos', 0.7755382657051086), ('muutos', 0.7696855664253235), ('lämpötiloi', 0.7514984011650085), ('lämpötilo', 0.7444837093353271), ('arvo_muutos', 0.7359239459037781), ('meri_lämpö#tila', 0.7352582812309265), ('lämpötilaero', 0.7276667952537537)]

-----
 savukaasu_rikinpoisto
4353 [('savukaasu', 0.8892187476158142), ('savukaasua_paloka#asu_poistoka#asu', 0.8

4378 [('näyttä', 0.8676709532737732), ('näyttätä', 0.849885106086731), ('näytti', 0.8493350148200989), ('näytä', 0.8477509617805481), ('näyttävä', 0.8324840068817139), ('näytää', 0.7972054481506348), ('näyttäisä', 0.793265700340271), ('tuntua', 0.7852351665496826), ('näytettää', 0.7710548639297485), ('näyt', 0.7617353200912476)]

-----
 lumivyöry
4379 [('lumivyötyä', 0.9999999403953552), ('lumivyötyä_maavyöry_kivivyöry', 0.870507538318634), ('lumivyöry_torjunta', 0.8509587049484253), ('maanvyöry', 0.6816624402999878), ('mutavyöry', 0.6816596984863281), ('kivivyöry', 0.6637446284294128), ('maavyöry', 0.6493229866027832), ('lumi', 0.616102397441864), ('lumimyrsky', 0.611015260219574), ('maanvyörymä', 0.6022315621376038)]

-----
 sieni#eläin
4380 [('sienieläin', 0.9112430214881897), ('vesieläin', 0.7135884761810303), ('sieni', 0.7041335105895996), ('norsueläin', 0.6983927488327026), ('merieläin', 0.6936642527580261), ('alkueläin', 0.6886497139930725), ('pussieläin', 0.6884257793426514), (